<html>
    <summary></summary>
         <div> <p></p> </div>
         <div style="font-size: 20px; width: 800px;"> 
              <h1>
               <left>Exponential Growth</left>
              </h1>
              <p><left>============================================================================</left> </p>
<pre>Course: ASU CBP Summer School 2025
Instructor: Dr. Douglas Shepherd
Contact Info: douglas.shepherd@asu.edu
Authors: Dr. Douglas Shepherd
</pre>
         </div>
    </p>

</html>

<details>
  <summary>Copyright info</summary>

```
Copyright 2018 Griffin Chure

Redistribution and use in source and binary forms, with or without modification, are permitted provided that the following conditions are met:

1. Redistributions of source code must retain the above copyright notice, this list of conditions and the following disclaimer.

2. Redistributions in binary form must reproduce the above copyright notice, this list of conditions and the following disclaimer in the documentation and/or other materials provided with the distribution.

3. Neither the name of the copyright holder nor the names of its contributors may be used to endorse or promote products derived from this software without specific prior written permission.

THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS" AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT HOLDER OR CONTRIBUTORS BE LIABLE FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY, OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.
```
<details>



<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/QI2lab/2025-CBP-SummerSchool/blob/main/Module4-BacteriaGrowth/M4A_Exponential_Growth.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/QI2lab/2025-CBP-SummerSchool/blob/main/Module4-BacteriaGrowth/M4A_Exponential_Growth.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

# Learning Objectives
In this lesson, we are going to discuss the topic of numerical simulation of bacteria population growth. After completing this lesson, you should be able to:

* Describe the concepts behind the exponential and logistic growth functions. 
* Derive and apply analytical solutions to the exponential and logistic growth functions.
* Numerically simulate the exponential and logistic growth functions.

# Contents
1. Forward-Euler algorithm
2. Exponential growth equation
3. Numerical integration of exponential growth
4. Logistic growth equation 
5. Numerical integration of logistic growth

In [ ]:
# Importing packages
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context('talk')

# **1. Introduction**

In class, we used some simple back-of-the-envelope estimates to figure out what sets the speed limit for bacterial growth. But what does this speed limit mean for the growth of populations? To answer this question, we turn to writing a simple ordinary differential equation.

Throughout the course, we will repeatedly mathematize our thinking using differential equations. For example, we can write a differential equation for a growing bacterial culture as 

$$
{dN \over dt} = r N(t) \tag{1},
$$

where $N$ is the the number of cells, $t$ is time (in whatever units we deem appropriate), and $r$ is the rate of bacterial growth. This differential equation can be very easily solved for $N(t)$ as

$$
N(t) = N_0 e^{rt} \tag{2},
$$

where $N_0$ is the intial number of starting cells. While getting to this result is relatively simple, this will certainly not be the case for every differential equation we write! It some cases, there may not even be a closed form solution. To solve such equations, we will have to turn to our computers to do the work for us. 

## The Forward-Euler Algorithm

There are seemingly countless numerical integrators available to you in nearly all computer programming languages. The simplest and easiest to code (although it has some limitations) is the Forward-Euler method. In this approach, we take very small steps forward in time, calculate the change in our quantity of interest, and then add that change to our result from the previous time step. 

Let's look back at Eq. 1, but rewrite it in a different way. We can say that the number of cells present at some time $t + \Delta t$ is

$$
N(t + \Delta t) = N(t) + r N(t) \Delta t. \tag{3}
$$

If we taking yet another step forward in time could similarly be stated as

$$
N(t + 2\Delta r) = N(t + \Delta t) + r N(t + \Delta t) \Delta t. \tag{4}
$$

At each time step, we are simply adding the number of cells added in that interval $\Delta t$ as was defined in our initial differential equation. 

But how do we choose our time step? For this approach to work, we must take steps forward in time that are sufficiently small such that no more than one event described in our model can take place. In the context of exponential growth, for example, we would not want to take steps forward in time that are larger than the growth rate where multiple divisions could occur. This requirement, known as the [Courant-Friedrichs-Lewy condition](https://en.wikipedia.org/wiki/Courant%E2%80%93Friedrichs%E2%80%93Lewy_condition) is important for many types of time-marching computer simulations.

# 2. The Exponential Growth Equation

First, we will create a numerical integrator for the case of simple exponential growth. To begin, we will define the biologically meaningful parameters. How does the rate `r` defined here related to the doubling time of the cells?

In [ ]:
# Define parameters for the exponential growth
N_0 = 1 # Initial number of cells 
r = 0.03 # Growth rate of cells in generations per minute
total_time = 150 # Total time of the integration in minutes.

To perform numerical integration, we have to select a time step. It should be small enough to properly integrate (see above), but not so small as to waste computational resources.

In [ ]:
# Define parameters for the numerical integrator
delta_t = 0.1 # time step in minutes
n_time_steps = int(total_time / delta_t) 

In the above code cell, we defined the size of our time step and figured out how many we will need to take given the total time of our experiment. While `n_time_steps` is an integer quantity (you can't have 1.3 steps!), simply dividing `total_time` by `delta_t` would result in a floating point number. By adding `int(...)`, I have forced the result to be an integer which will prove be important for us shortly. 

We now have just about everything we need to begin our integration. The one last things we need to do is set our initial condition and then loop through all of the time steps. As we would like to keep track of the number of cells as a function of time, we'll make a vector full of zeros that has the same length as `n_time_steps`.

In [ ]:
# Set up the empty vector where we will store the number of cells at time t.
N_t = np.zeros(n_time_steps)

# Set the initial condition by indexing the array. 
N_t[0] = N_0

We're now ready to do the integration! Starting at the second time point (index `1`), we'll calculate how many cells were added in a single time step and update our storage vector `N_t`. 

In [ ]:
# Loop through each time step
# We say range(1, N) instead of range(N) to begin at index 1
for t in range(1, n_time_steps): 
    
    # Calculate the change in the number of cells. 
    dN = N_t[t - 1] * r * delta_t
    
    # Update the number of cells at the current time point
    N_t[t] = N_t[t - 1] + dN    

Because of the simplicity of the differential equation, this code should run almost instantaneously on your machine. Let's plot our result to see if we got exponential growth out of it all.

In [ ]:
# Set up the time array for the x axis
# np.arange will arrange numbers from 0 to the total time, taking steps of dt
time_range = np.arange(0, total_time, delta_t)


# Plot our calculation as a blue line
plt.plot(time_range, N_t, 'b-', label='numerical integration')

# Add an appropriate label and legend
plt.xlabel('time [min]')
plt.ylabel('number of cells')
plt.legend()

To ensure that we got the correct result, we can plot a subset of points from the numerical integration over our analytical solution in Eq. 2. 

In [ ]:
# Compute the analytical solution.
solution = N_0 * np.exp(r * time_range)

# Plot every 50 points from the numerical integration.
plt.plot(time_range[::50], N_t[::50], 'bo', label='numerical integration')

# Plot the analytical solution as a red line. 
plt.plot(time_range, solution, 'r-', label='analytical solution')

# Add appropriate labels. 
plt.xlabel('time [min]')
plt.ylabel('number of cells')
plt.legend()

It looks like our simple numerical integrator works as advertised and converges to the analytical result. While valuable, we may not always have an analytical result to verify the numerical result with.

As we discussed, the stability of this specific numerical approach is dependent on at most one "event" occuring during the chosen time step. What happens if we violate this by choosing a longer time step, such as `delta_t=10`?  

In [ ]:
# Set the new parameters
delta_t_long = 10 
n_time_steps_long = int(total_time / delta_t_long)

# Set the storage vector so we don't rewrite our correct approach
N_t_long = np.zeros(n_time_steps_long)
N_t_long[0] = N_0

# Loop through each time step
for t in range(1, n_time_steps_long): 
    
    # Calculate the change in the number of cells. 
    dN = N_t_long[t - 1] * r * delta_t_long
    
    # Update the number of cells at the current time point
    N_t_long[t] = N_t_long[t - 1] + dN    

Let's visualize the numerical result and analytical solution.

In [ ]:
# Plot the analytical solution
plt.plot(time_range, solution, 'r-', label='analytical solution')

# Plot every fifth point from the long time step integration
time_range_long = np.arange(0, total_time, delta_t_long)
plt.plot(time_range_long, N_t_long, 'go', label='Δt = 10 min')

# Add labels as usual
plt.xlabel('time [min]')
plt.ylabel('number of cells')
plt.legend()

After the first few time points, where there are fewer "events", the numerical solution deviates from the analytical solution. Try exploring other `delta_t` values on your own.

# 3. The Logistic Growth Equation

What happens as the bacteria continue to grow? If we take the limit of Eq. 1 as time grows large, then the number of bacteria goes to infinity! We know this isn't true, suggesting there must be some sort of feedback mechanism. The feedback could be nutrient availability, limited volume, or other potential restrictions. Ultimately, all of the restrictions determine the "carrying capacity" of the population of interesting. This concept is nearly 200 years old, attributed to [François Verlhust](https://en.wikipedia.org/wiki/Pierre_Fran%C3%A7ois_Verhulst) in 1838. Mathematically, we can express such a growth process as

$$
{dN \over dt} = rN(t)\left(1 - {N(t) \over K}\right) \tag{5},
$$

where $K$ is the carrying capacity. From looking at this equation, we can see that as the total number of cells gets closer and closer to $K$, the number of cells added during a particular time step gets smaller and smaller until the carrying capacity is met, $N = K$.

Unlike Eq. 1, this differential equation is much less trivial to solve, although an analytical solution exists. Let's put our skills of numerical integration to the test and integrate Eq. 5 using a few different values for the carrying capacity.

In [ ]:
# Set the carrying capacity:
K = [75, 1.5E2, 3E2]

# Extend the total time of the integration
delta_t = 0.1
total_time = 500
time_range = np.arange(0, total_time, delta_t)
n_time_steps = int(total_time / delta_t)
# Set the storage vector so we don't rewrite our correct approach
N_t = np.zeros((len(K), n_time_steps))
N_t[:, 0] = N_0

# Loop through each carrying capacity
for k in range(len(K)): 
    
    # Loop through each time step.
    for t in range(1, n_time_steps): 
    
        # Calculate the change in the number of cells. 
        dN = N_t[k, t - 1] * r * delta_t * (1 - N_t[k, t - 1] / K[k])
    
        # Update the number of cells at the current time point
        N_t[k, t] = N_t[k, t - 1] + dN
         
# Loop through the carrying capacities and plot every 100th point.
for i in range(len(K)):
    plt.plot(time_range[::100], N_t[i, ::100], '.', label='K = ' + str(K[i]))
    
# Add appropriate labels and legends. 
plt.xlabel('time  [min]')
plt.ylabel('number of cells')
plt.legend() 

While the rate of growth is the same for all three cases, the imposed carrying capacity causes growth to cease at different times. We also see that growth slows down gradually as the carrying capacity is approached, rather than simply rocketing to the carrying capacity at the maximal rate. We could tune this logistic growth even further to account for the senecense of cells, reducing the number of individuals below the carrying capacity.

As mentioned above, there is an analytical solution for the logistic growth model given in Eq. 5, although it is less straightforward to solve than Eq. 1. For posterity's sake, we will solve the logistic growth differential equation and plot it over the solution found through numerical integration. By separation of variables, we can rewrite Eq. 5 as

$$
rdt = {dN \over N\left(1 - {N \over K}\right)} \tag{6}
$$

We can now integrate both sides, 

$$
\int r dt = \int {1 \over N \left(1 - {N \over K}\right)} dN, \tag{7}
$$

but the integral on the rightside of Eq. 7 is quite difficult to solve. As a first step, we can use a [partial fraction decomposition](https://en.wikipedia.org/wiki/Partial_fraction_decomposition) to break the integrand into two pieces,

$$
{1 \over rN\left(1 - {N \over K}\right)} = {A \over N} + {B \over 1 - {N  \over K}}, \tag{8}
$$

where $A$ and $B$ are unknown constants. We can clear the fractions from Eq. 8 as

$$
1 = A\left(1 - {N \over K}\right) + BN.  \tag{9}
$$

This formulation allows us to identify the constants $A$ and $B$ by looking at the extrema. When $N = 0$, Eq. 9 becomes

$$
1 = A\times(1 - 0) + B\times 0 \tag{10},
$$

meaning that

$$
A = 1. \tag{10}
$$

Conversely, when $N = K$, we find the value of $B$ to be 

$$
B = {1 \over K} \tag{11}.
$$

The right-hand integral in Eq. 7 can now be rewritten as

$$
\int {1 \over rN\left(1 - {N \over K}\right)} = \int {1 \over N} dN + \int {{1 \over K} \over 1 - {N \over K}} dN. \tag{12}
$$

Using u-substitution for the right-hand integral in Eq. 12, we can now return to Eq. 7 and integrate both sides, arriving at

$$
rt + C = \ln N - \ln \left(1 - {N \over K}\right) \tag{13},
$$

where $C$ is a constant of integration. Exponentiating both sides of Eq. 13 ands solving for $N$ yields

$$
N = {e^C e^{rt} \over 1 + {e^{c}e^{rt} \over K}} \tag{14}.
$$

We can massage this into a more friendly form by multiplying the top and bottom by $Ke^{-rt}$ to arrive at

$$
N = {e^C K \over Ke^{-rt} + e^C}. \tag{15}
$$

We are still left with our constant of integration $C$ hanging around in our equation. We can determine this constant by noticing that at our initial time $t = 0$, the number of cells $N_0$ is 

$$
N_0 = {e^C K \over K + e^C}. \tag{16},
$$

which, with some rearrangement, gives us

$$
e^C = {N_0 K \over K - N_0}. \tag{17}
$$

We can now plug this back into Eq. 15 and do some algebra to arrive at our analytical solution for the number of cells at time $t$ as

$$
N(t) = {N_0 K \over e^{-rt}(K - N_0) + N_0}. \tag{18}
$$

Though still manageable, this is not nearly as trivial as finding the analytical solution for unrestricted exponential growth given as Eq. 1! For a quick sanity check, we can plot this analytical solution over the results from our numerical integration.

In [ ]:
# Solve the analytical solution for our three carrying capacities.  
solution = np.zeros((len(K), n_time_steps))
for i in range(len(K)):
    solution[i, :] = N_0 * K[i] / (np.exp(-r * time_range) * (K[i] - N_0) + N_0)
    
# Define the colors so they match up. 
colors = ['green', 'orange', 'blue']

# Set up a figure that's a bit bigger
fig = plt.figure(figsize=(9, 5))

# Plot the analytical solutions and numerical integrations 
for i in range(len(K)):
    # Analytical solution
    plt.plot(time_range, solution[i,:], color=colors[i], 
             label='solution, K = ' + str(K[i]))
    
    # Numerical integration
    plt.plot(time_range[::200], N_t[i, ::200], '.', markersize=10, color=colors[i],
            label='integration, K = ' + str(K[i]))
   
# Add axis labels and a legend.
plt.xlabel('time [min]')
plt.ylabel('number of cells')
plt.legend()

# 4. Conclusion

Again, we see that our simple numerical integration gave us the same result as our analytical solution. While having an analytical solution is often very desireable, it's not always necessary. It's up to you to decide when you need to grind through the math to get the solution versus using numerical integration.

While we used the Forward-Euler method to perform these integrations, this method is rather unstable and there are some differential equations for which it simply will not work. In your research, you will want to use integration packages in the programming language of your choice (such as [`scipy.integrate.odeint`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.odeint.html) for Python or [`ode45`](https://www.mathworks.com/help/matlab/ref/ode45.html) in MATLAB). These methods, though more complicated, are far faster and more robust to pathological functions that the Forward-Euler method.